### Preprocessing sunrise and sunset time to encode it as daytime or nighttime

In [1]:
import requests
import pandas as pd
import dateutil.parser

In [2]:
#1.  Get data from the api

In [3]:
url = 'https://archive-api.open-meteo.com/v1/archive'

params_daily_dict ={
        "latitude":"51.5085", #London latitude - should remain hardcoded
        "longitude":"-0.1780971", #London longitude -should remain hardcoded
        "start_date":"2023-08-20", #could be defined in .env and used in the other files
        "end_date":"2023-08-23", #could be defined in .env and used in the other files
        "timezone":"Europe/London", #Europe/London - specific to this api
        "daily":"sunrise,sunset" # specific to this api
}

daily_weather_response = requests.get(
        'https://archive-api.open-meteo.com/v1/archive',
        params=params_daily_dict).json()

sunset_time = daily_weather_response["daily"]["sunset"]
sunrise_time = daily_weather_response["daily"]["sunrise"]

In [4]:
daily_data = pd.DataFrame()

In [5]:
daily_data["sunrise"] = sunrise_time
daily_data["sunset"] = sunset_time

In [6]:
daily_data.head()

,sunrise,sunset
0,2023-08-20T05:54,2023-08-20T20:14
1,2023-08-21T05:55,2023-08-21T20:12
2,2023-08-22T05:57,2023-08-22T20:10
3,2023-08-23T05:58,2023-08-23T20:07


In [7]:
#1bis transform the time into datetime

In [8]:
def date_parser(time):
    date_parser = dateutil.parser.isoparse(time)
    return date_parser

In [9]:
daily_data["sunrise_datetime"] = daily_data.apply(lambda x: date_parser(x["sunrise"]), axis = 1)

In [10]:
daily_data["sunset_datetime"] = daily_data.apply(lambda x: date_parser(x["sunset"]), axis = 1)

In [11]:
daily_data

,sunrise,sunset,sunrise_datetime,sunset_datetime
0,2023-08-20T05:54,2023-08-20T20:14,2023-08-20 05:54:00,2023-08-20 20:14:00
1,2023-08-21T05:55,2023-08-21T20:12,2023-08-21 05:55:00,2023-08-21 20:12:00
2,2023-08-22T05:57,2023-08-22T20:10,2023-08-22 05:57:00,2023-08-22 20:10:00
3,2023-08-23T05:58,2023-08-23T20:07,2023-08-23 05:58:00,2023-08-23 20:07:00


In [12]:
daily_data["sunset_datetime"] < daily_data["sunrise_datetime"]

0    False
1    False
2    False
3    False
dtype: bool

In [13]:
#2.  Idea: 
    ## 2.1  hours before sunrise time should be encoded as "nighttime"
    ## 2.2 hours after sunset time should be encoded as "nightime"
    ## 2.3 hours between sunrise and sunset should be encoded as "daytime"
    ## 2.4 the hour when sun rises should be encoded as "nightime"
    ## 2.5 the hour when the sun sets shouldbe encoded as "daytime"

In [14]:
def daytime_encoding(time_test, sunrise_datetime, sunset_datetime):
    if time_test < sunrise_datetime:
        daytime_encoding = "nighttime"
    elif time_test >= sunrise_datetime and time_test < sunset_datetime:
        daytime_encoding = "daytime"
    else:
        daytime_encoding = "nighttime"
    return daytime_encoding

In [ ]:
#3. Add encoded data to an additional column in the weather_data df

In [ ]:
daily_data["encoding"] = daily_data.apply(lambda x: daytime_encoding(x["test_time"], x["sunrise_datetime"], x["sunset_datetime"]), axis = 1)

In [ ]:
daily_data